In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
import nltk
from pymagnitude import *

In [2]:
X = []
y = []
with open("/Users/zetong/intents.csv", mode = 'r', encoding = 'ascii', errors = 'ignore') as csvfile:
    intents = pd.read_csv(csvfile)
    X = list(intents["utterances"])
    y = list(intents["labels"])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
X = np.asarray(X)

In [3]:
le.classes_

array(['commonQ.assist', 'commonQ.bot', 'commonQ.how',
       'commonQ.just_details', 'commonQ.name', 'commonQ.not_giving',
       'commonQ.query', 'commonQ.wait', 'contact.contact',
       'faq.aadhaar_missing', 'faq.address_proof',
       'faq.application_process', 'faq.apply_register',
       'faq.approval_time', 'faq.bad_service',
       'faq.banking_option_missing', 'faq.biz_category_missing',
       'faq.biz_new', 'faq.biz_simpler', 'faq.borrow_limit',
       'faq.borrow_use'], dtype='<U26')

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [12]:
vectors = Magnitude("/Users/zetong/Downloads/glove.840B.300d.magnitude")

In [8]:
MAX_SEQ_LEN = 50
i = tf.keras.layers.Input(shape=(MAX_SEQ_LEN, vector.dim))
Bidir_LSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation="tanh", return_sequences=True), merge_mode="concat",)(i)
maxpool = tf.keras.layers.GlobalMaxPooling1D()(Bidir_LSTM)
hidden = tf.keras.layers.Dense(32)(maxpool)
output = tf.keras.layers.Dense(1, activation="softmax")(hidden)
model = tf.keras.Model(inputs=i, outputs=output)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50, 300)]         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 64)            85248     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 87,361
Trainable params: 87,361
Non-trainable params: 0
_________________________________________________________________


In [13]:
def make_dataset(xarr, yarr):
    dataset = tf.data.Dataset.from_tensor_slices((xarr, yarr)).repeat()
    
    def _process_string(x):

        # x is numpy array
        def _pad_zeros(x, MAX_SEQ_LEN):
            if x.shape[0] >= MAX_SEQ_LEN:
                return x[0:MAX_SEQ_LEN, :]
            else:
                return np.concatenate(
                    (x, np.zeros((MAX_SEQ_LEN - x.shape[0], x.shape[1]))), axis=0
                )

        x = x.numpy().decode()
        x = word_tokenize(x)
        if len(x) != 0:
            x = vectors.query(x)
            x = _pad_zeros(x, MAX_SEQ_LEN)
        else:
            x = np.zeros((MAX_SEQ_LEN, vectors.dim))
        return x
    
    def _process_datapair(X, y):
        X = tf.py_function(_process_string, [X], tf.float32)
        X.set_shape([MAX_SEQ_LEN, vectors.dim])
        y.set_shape([])
        return X, y
    
    dataset = dataset.map(_process_datapair)
    return dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(batch_size)

In [14]:
train = make_dataset(X_train, y_train)

In [15]:
train

<DatasetV1Adapter shapes: ((?, 50, 300), (?,)), types: (tf.float32, tf.int64)>

In [18]:
model.fit(train, steps_per_epoch=1, epochs=1)

W0909 13:51:22.951592 4661499328 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0909 13:51:25.245700 123145448779776 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 13:51:25.292820 123145450389504 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 13:51:25.337610 123145448779776 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 13:51:25.379935 123145448779776 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0909 13:51:25.403928 123145448779776 backprop.py

1/1 [==============================] - 7s 7s/step - loss: -162.9156 - acc: 0.0000e+00
